# Proof Of Concept - Corpus Collection

## Importing downloaded data on LinkedIn posts from kaggle

Kaggle Link: https://www.kaggle.com/datasets/shreyasajal/linkedin-influencers-data

About:

This dataset contains LinkedIn Influencers' post details and other details(post dependent as well as independent) per post. This dataset can be used to analyze LinkedIn reach based on post content and related account details.

This dataset is great for Exploratory Data Analysis and NLP tasks.

The data was scraped using BeautifulSoup and Selenium.Last updated on 15th Feb,2021

In [1]:
import pandas as pd

original_data = pd.read_csv("../data/linkedIn_data.csv")

C:\Users\mujta\AppData\Local\Temp\ipykernel_39444\2105851821.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  original_data = pd.read_csv("../data/linkedIn_data.csv")


## Displaying data rows and its column properties

In [2]:
original_data.head()

,Unnamed: 0,name,headline,location,followers,connections,about,time_spent,content,content_links,media_type,media_url,num_hashtags,hashtag_followers,hashtags,reactions,comments,views,votes
0,0,Nicholas Wyman,CEO IWSI Group,NaN,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,1 day ago,Robert Lerman writes that achieving a healthy...,[['https://www.linkedin.com/in/ACoAAACy1HkBviR...,article,['https://www.urban.org/urban-wire/its-time-mo...,4,0,"[['#workbasedlearning', 'https://www.linkedin....",12,1,NaN,NaN
1,1,Nicholas Wyman,CEO IWSI Group,NaN,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,1 week ago,"National disability advocate Sara Hart Weir, ...",[['https://www.linkedin.com/in/ACoAAAHsfJgBb7_...,NaN,[],0,0,[],11,0,NaN,NaN
2,2,Nicholas Wyman,CEO IWSI Group,NaN,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,2 months ago,NaN,[],NaN,[],0,0,[],15,0,NaN,NaN
3,3,Nicholas Wyman,CEO IWSI Group,NaN,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,2 months ago,Exploring in this months Talent Management & H...,[['https://www.linkedin.com/in/ACoAAAADlGIBLfn...,article,['https://www.tlnt.com/apprenticeships-that-br...,4,0,"[['#careerplanning', 'https://www.linkedin.com...",44,0,NaN,NaN
4,4,Nicholas Wyman,CEO IWSI Group,NaN,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,2 months ago,I count myself fortunate to have spent time wi...,[['https://www.linkedin.com/in/ACoAABhNxDUB9IX...,article,['https://gritdaily.com/the-legacy-of-verified...,3,0,"[['#verifiedresumes', 'https://www.linkedin.co...",22,2,NaN,NaN


In [3]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34012 entries, 0 to 34011
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         34012 non-null  int64  
 1   name               34012 non-null  object 
 2   headline           34012 non-null  object 
 3   location           31740 non-null  object 
 4   followers          33970 non-null  float64
 5   connections        25713 non-null  object 
 6   about              34012 non-null  object 
 7   time_spent         34011 non-null  object 
 8   content            31996 non-null  object 
 9   content_links      34012 non-null  object 
 10  media_type         26779 non-null  object 
 11  media_url          34012 non-null  object 
 12  num_hashtags       34012 non-null  int64  
 13  hashtag_followers  34012 non-null  int64  
 14  hashtags           34012 non-null  object 
 15  reactions          34012 non-null  int64  
 16  comments           340

## We will work on content column (posts) and will sample approximately 1600 contents (posts) for annotation as the original size is 34000

## Sample Content (Post) and example of manual annotation

In [10]:
print(original_data.loc[2010,'content'])

Oops...Before you fall in love (with technology)make sure ‘they are true-blue...’ Turns out your Apple Watch has been cheating on you! According to a massive new study just published, turns out that the pulse sensor in Apple Inc.’s watch helped detect a heart-rhythm disorder in a small number of users but may have caused false alarms for others.  As is true when it comes to physiology—and pathophysiology—our body systems are always searching for homeostasis. Meaning, our bodies are always trying to function normally and within a small band of performance. Yet normal variation exists in performance that are no big deal... Like your heart doesn’t beat exactly the same all the time. And it’s fine. Normal variation! But the Apple Watch might be mis-characterize these beats, and label them as more significant than they are. This is called a false positive! So you need to watch out for being over-treated or mis-treated if you seek care for this ‘normal’ variations. Selected Excerpt: “The stu

### Now we will label above post with one of the following temporary decided labels:

1. Professional Growth: Includes achievements, job advancements, networking advice,
and hiring posts.

2. Industry & Culture: Covers trends, lessons, company culture, diversity, and inclusion.
 
3. Engagement & Learning: Focuses on interactive content, educational resources, and
humor

Due to the nature of the post which is themed towards learning about Apple technology, we will classify it in the "Engagement & Learning" label. Similarly, once we finalize the labels and annotation schema we will then annotate these instances.

## Following is an example of how we will be sampling posts from the original data

Using random state for reproducibility

In [15]:
sampled_data = original_data.sample(n=5, random_state=42) 

In [16]:
sampled_data.head()

,Unnamed: 0,name,headline,location,followers,connections,about,time_spent,content,content_links,media_type,media_url,num_hashtags,hashtag_followers,hashtags,reactions,comments,views,votes
11313,11313,Lynne Everatt,Author of The 5-Minute Recharge and Acts of Fr...,"['Oakville,', 'Ontario,', 'Canada']",301184.0,500+,"If you were to meet me at party, I’d want you ...",3 months ago,The best refuge during a pandemic...,[],image,['https://media-exp1.licdn.com/dms/image/C4E22...,0,0,[],102,4,NaN,NaN
23534,23534,Jody Padar,Co-Author Botkeeper For Dummies at Botkeeper,"['Prospect', 'Heights,', 'Illinois,', 'United'...",661723.0,500+,About my Firm:Envision Your Healthy Financial ...,9 months ago,Recently I had the pleasure of speaking with ...,[['https://www.linkedin.com/feed/hashtag/?keyw...,image,['https://media-exp1.licdn.com/dms/image/C4D22...,7,0,"[['#TheAccidentalAccountant', 'https://www.lin...",12,1,NaN,NaN
3386,3386,"Bertalan Meskó, MD, PhD",Director of The Medical Futurist Institute (Ke...,['Hungary'],263346.0,500+,"Dr. Bertalan Mesko, PhD is The Medical Futuris...",2 years ago,Yet another example for how a very simple digi...,[['https://www.linkedin.com/feed/hashtag/?keyw...,image,['https://media-exp1.licdn.com/dms/image/C5622...,9,0,"[['#themedicalfuturist', 'https://www.linkedin...",220,14,NaN,NaN
12659,12659,Justin Bariso,Author at EQ Applied | Management Thinker an...,"['New', 'York', 'City', 'Metropolitan', 'Area']",219717.0,500+,I write weekly on the topic of emotional intel...,4 years ago,Bonus column for Tuesday.,[],article,['http://www.inc.com/justin-bariso/this-ceo-ca...,0,0,[],41,3,NaN,NaN
8037,8037,James Altucher,"Founder at ""The James Altucher Show"" podcast","['New', 'York', 'City', 'Metropolitan', 'Area']",1268865.0,NaN,"James is a Top 10 Linkedin Influencer, prolifi...",2 years ago,"Mostly, I was a criminal. Unless I was excited...",[],article,['https://jamesaltucher.com/2016/12/day-dream-...,0,0,[],73,5,NaN,NaN


In [17]:
sampled_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 11313 to 8037
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5 non-null      int64  
 1   name               5 non-null      object 
 2   headline           5 non-null      object 
 3   location           5 non-null      object 
 4   followers          5 non-null      float64
 5   connections        4 non-null      object 
 6   about              5 non-null      object 
 7   time_spent         5 non-null      object 
 8   content            5 non-null      object 
 9   content_links      5 non-null      object 
 10  media_type         5 non-null      object 
 11  media_url          5 non-null      object 
 12  num_hashtags       5 non-null      int64  
 13  hashtag_followers  5 non-null      int64  
 14  hashtags           5 non-null      object 
 15  reactions          5 non-null      int64  
 16  comments           5 non-nul